In [ ]:
# select cuda device
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import tensorflow as tf
print(tf.__version__)

import numpy as np
from glob import glob as glb
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#this function write tfrecords
# input_ex is input of shape shape[0], shape[1], 3
# out_ex is output of shape[0], shape[1], 2
shape = [1024, 1024]
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def writeTfrecord(filename_list, record_filename):
    writer = tf.io.TFRecordWriter(record_filename)
    for filename in filename_list:
        split = filename.split("/")[-1].split("_")
        df = pd.read_csv(filename, index_col=None, header=None)
        data = df[[0,1,3,4]]
        out_ex = np.empty([shape[0], shape[1], 2]) 
        out_ex[:,:,0] = -1 + (2 * ((data.values[:,2].reshape(shape[0], shape[1]) - min_x) / (max_x - min_x)))
        out_ex[:,:,1] = -1 + (2 * ((data.values[:,3].reshape(shape[0], shape[1]) - min_y) / (max_y -min_y)))
        
        input_ex = np.empty([shape[0], shape[1], 3])
        input_ex[:,:,0] = np.isnan(out_ex[:,:,0])*1
        input_ex[:,[0,-1],0] = 1
        input_ex[:,:,1] = out_ex[0,:,0]
        input_ex[:,:,2] = out_ex[0,:,1]
        where_are_NaNs = np.isnan(out_ex)
        out_ex[where_are_NaNs] = 0
        out_ex[:,[0,-1],:]=0

        input_ex = np.float32(input_ex)
        input_ex = input_ex.reshape([1, shape[0]*shape[1]*3])
        input_ex = input_ex.tobytes()

        out_ex = np.float32(out_ex)
        out_ex = out_ex.reshape([1,shape[0]*shape[1]*2])
        out_ex = out_ex.tobytes()

        feature={'input_ex': _bytes_feature(input_ex), 'out_ex': _bytes_feature(out_ex)}
        example = tf.train.Example(features=tf.train.Features(feature = feature))
        writer.write(example.SerializeToString()) 
    writer.close()

In [ ]:
# read tfrecord and check for mistakes
tr_ds = tf.data.TFRecordDataset(["../../data/subDomain_train_1e-5.tfrecords"])
def parse_example(example_proto):
    # Create a description of the features.
    features={'input_ex':tf.io.FixedLenFeature([],tf.string), 'out_ex':tf.io.FixedLenFeature([],tf.string)}
    # Parse the input tf.Example proto using the dictionary above.
    return tf.io.parse_single_example(example_proto, features)

tr_parse_ds = tr_ds.map(parse_example)
dataset_iterator = iter(tr_parse_ds)

current_example = dataset_iterator.next()
input_ex = tf.io.decode_raw(current_example['input_ex'], tf.float32)
out_ex = tf.io.decode_raw(current_example['out_ex'], tf.float32)
input_ex = tf.reshape(input_ex, [shape[0], shape[1], 8])
out_ex = tf.reshape(out_ex, [shape[0], shape[1], 2])
#input_ex = tf.cast(input_ex, dtype=tf.float32)
#out_ex = tf.cast(out_ex, dtype=tf.float32)
print("Data statistics")
print(np.amax(out_ex[:,:,0]), np.amin(out_ex[:,:,0]), np.mean(np.abs(out_ex[:,:,0])))
print(np.amax(out_ex[:,:,1]), np.amin(out_ex[:,:,1]), np.mean(np.abs(out_ex[:,:,1])))


In [ ]:
#this function read tfrecord
def decode_example(record, shape=(128,128)):
    record['input_ex'] = tf.io.decode_raw(record['input_ex'], tf.float32)
    record['out_ex'] = tf.io.decode_raw(record['out_ex'], tf.float32)
    record['input_ex'] = tf.reshape(record['input_ex'], [shape[0], shape[1], 9])
    record['out_ex'] = tf.reshape(record['out_ex'], [shape[0], shape[1], 2])
    record['input_ex'] = tf.cast(record['input_ex'], dtype=tf.float32)
    record['out_ex'] = tf.cast(record['out_ex'], dtype=tf.float32)
    return record['input_ex'], record['out_ex']

def get_dataset(filename, input_channel=9):
    tr_ds = tf.data.TFRecordDataset(filename)
    tr_parse_ds = tr_ds.map(parse_example)
    tr_decoded_ds = tr_parse_ds.map(decode_example)
    train_ds =  tr_decoded_ds.batch(8)
    return train_ds

In [ ]:
# to pass in model
train_ds = get_dataset(filename)
model.train(train_ds)